In [ ]:
pip install opencv-python dlib face_recognition


In [ ]:
import cv2
import face_recognition
import os

# Function to encode faces from the training dataset
def encode_faces(dataset_path):
    known_face_encodings = []
    known_face_names = []
    
    for image_name in os.listdir(dataset_path):
        image_path = os.path.join(dataset_path, image_name)
        image = face_recognition.load_image_file(image_path)
        encoding = face_recognition.face_encodings(image)[0]
        known_face_encodings.append(encoding)
        known_face_names.append(os.path.splitext(image_name)[0])
    
    return known_face_encodings, known_face_names

# Function to recognize faces in a given image
def recognize_faces(image_path, known_face_encodings, known_face_names):
    image = face_recognition.load_image_file(image_path)
    face_locations = face_recognition.face_locations(image)
    face_encodings = face_recognition.face_encodings(image, face_locations)

    recognized_faces = []
    
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"

        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        best_match_index = min(range(len(face_distances)), key=lambda i: face_distances[i])
        
        if matches[best_match_index]:
            name = known_face_names[best_match_index]

        recognized_faces.append(name)

        cv2.rectangle(image, (left, top), (right, bottom), (0, 0, 255), 2)
        cv2.putText(image, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
    
    return image, recognized_faces

# Main code
dataset_path = 'path/to/your/dataset'  # Path to the folder containing training images
known_face_encodings, known_face_names = encode_faces(dataset_path)

image_path = 'path/to/image.jpg'  # Path to the image you want to recognize faces in
recognized_image, recognized_faces = recognize_faces(image_path, known_face_encodings, known_face_names)

print(f"Recognized faces: {recognized_faces}")

# Display the image with recognized faces
cv2.imshow('Recognized Faces', recognized_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
